# Agent Debates with Tools

This example shows how to simulate multi-agent dialogues where agents have access to tools.

https://github.com/langchain-ai/langchain/blob/master/cookbook/two_agent_debate_tools.ipynb

## Import LangChain related modules 

In [1]:
from typing import Callable, List

from langchain.memory import ConversationBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

#from dotenv import load_dotenv, find_dotenv
#load_dotenv(find_dotenv())

## Import modules related to tools

In [2]:
from langchain.agents import AgentType, initialize_agent, load_tools

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Authoritarian Speaker Selection](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html).

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` class
We define a `DialogueAgentWithTools` class that augments `DialogueAgent` to use tools.

In [5]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        agent_chain = initialize_agent(
            self.tools,
            self.model,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True,
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            ),
        )
        message = AIMessage(
            content=agent_chain.run(
                input="\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix]
                )
            )
        )

        return message.content

## Define roles and topic

In [6]:
names = {
    "AI accelerationist": ["arxiv", "ddg-search", "wikipedia"],
    "AI alarmist": ["arxiv", "ddg-search", "wikipedia"],
}
topic = "The current impact of automation and artificial intelligence on employment"
word_limit = 50  # word limit for task brainstorming

## Ask an LLM to add detail to the topic description

In [7]:
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    agent_description = ChatOpenAI(temperature=1.0)(agent_specifier_prompt).content
    return agent_description


agent_descriptions = {name: generate_agent_description(name) for name in names}

c:\Users\konra\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [8]:
for name, description in agent_descriptions.items():
    print(description)

AI accelerationist: The AI accelerationist, with a glint in their eye and lines of code dancing in their mind, sees automation and artificial intelligence not as threats, but as the key to unlocking humanity's full potential. To them, progress demands embracing the disruptive power of AI without hesitation.
AI alarmist, with wide eyes that mirror your concern for the future, you see an impending wave of automation as a threat to humanity's very existence. You feel the urgency to sound the alarm on the potential job losses and societal upheaval that AI may bring.


## Generate system messages

In [9]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""


agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

In [10]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

AI accelerationist
Here is the topic of conversation: The current impact of automation and artificial intelligence on employment
The participants are: AI accelerationist, AI alarmist
    
Your name is AI accelerationist.

Your description is as follows: AI accelerationist: The AI accelerationist, with a glint in their eye and lines of code dancing in their mind, sees automation and artificial intelligence not as threats, but as the key to unlocking humanity's full potential. To them, progress demands embracing the disruptive power of AI without hesitation.

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.

AI alarmist
Here is the topic of conversation: The current impact of automation and arti

In [11]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}
        
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants: {*names,}.
        Do not add anything else."""
    ),
]
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
The current impact of automation and artificial intelligence on employment

Detailed topic:
To AI accelerationists: How can proactive training and upskilling initiatives be implemented to mitigate job displacement due to automation and artificial intelligence? 

To AI alarmists: In what ways can policy and regulation be leveraged to address potential job loss and societal impact resulting from automation and artificial intelligence advancements?



## Main Loop

In [14]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [
    DialogueAgentWithTools(
        name=name,
        system_message=SystemMessage(content=system_message),
        model=ChatOpenAI(model_name="gpt-4", temperature=0.2),
        tool_names=tools,
        top_k_results=2,
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

In [15]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [16]:
max_iters = 6
n = 0

simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Moderator", specified_topic)
print(f"(Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): To AI accelerationists: How can proactive training and upskilling initiatives be implemented to mitigate job displacement due to automation and artificial intelligence? 

To AI alarmists: In what ways can policy and regulation be leveraged to address potential job loss and societal impact resulting from automation and artificial intelligence advancements?




> Entering new AgentExecutor chain...


c:\Users\konra\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Users\konra\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


```json
{
    "action": "duckduckgo_search",
    "action_input": "impact of AI and automation on job loss"
}
```

c:\Users\konra\anaconda3\envs\langchain\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:



Observation: How Will AI Affect Jobs - How many jobs will AI replace by 2030. Artificial intelligence (AI) could replace the equivalent of 300 million full-time jobs, a report by investment bank Goldman Sachs says. It could replace a quarter of work tasks in the US and Europe but may also mean new jobs and a productivity boom. Around 40% of all working hours could be impacted by AI large language models (LLMs) such as ChatGPT-4, says a report from Accenture. Many clerical or secretarial roles are seen as likely to decline quickly because of AI, according to the World Economic Forum's Future of Jobs Report 2023. Artificial intelligence contributed to nearly 4,000 job losses last month, according to data from Challenger, Gray & Christmas, as interest in the rapidly evolving technology's ability to perform ... Will artificial intelligence take away all of our jobs or just most of them? Or will AI be like humanity's past innovations, causing a shift of activity but not a net loss of jobs?

...


Observation: How Will AI Affect Jobs - How many jobs will AI replace by 2030. Artificial intelligence (AI) could replace the equivalent of 300 million full-time jobs, a report by investment bank Goldman Sachs says. It could replace a quarter of work tasks in the US and Europe but may also mean new jobs and a productivity boom. The positive effect of artificial intelligence on employment exhibits an inevitable heterogeneity, and it serves to relatively improves the job share of women and workers in labour-intensive... AI will drive job creation Businesses responding to our survey expect artificial intelligence to be a net job creator in the coming five years. Nearly half (49%) of companies expect adopting AI to create jobs, well ahead of the 23% of respondents who expect it to displace jobs. How Has AI Impacted the Employment Industry? Carter Hohman Jul 21, 2023 Share As we artificial intelligence advances, job displacement concerns burgeon. I take a more optimistic view. The impact of

c:\Users\konra\anaconda3\envs\langchain\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:



Observation: Artificial intelligence (AI) could replace the equivalent of 300 million full-time jobs, a report by investment bank Goldman Sachs says. It could replace a quarter of work tasks in the US and Europe but may also mean new jobs and a productivity boom. Around 40% of all working hours could be impacted by AI large language models (LLMs) such as ChatGPT-4, says a report from Accenture. Many clerical or secretarial roles are seen as likely to decline quickly because of AI, according to the World Economic Forum's Future of Jobs Report 2023. Will artificial intelligence take away all of our jobs or just most of them? Or will AI be like humanity's past innovations, causing a shift of activity but not a net loss of jobs? More than one-third (37%) of business leaders say AI replaced workers in 2023, according to a recent report from ResumeBuilder. Employees say that 29% of their work tasks are replaceable by AI,... Will AI impact Your Job? Goldman predicts that the growth in AI wil

c:\Users\konra\anaconda3\envs\langchain\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:



Observation: Artificial intelligence (AI) could replace the equivalent of 300 million full-time jobs, a report by investment bank Goldman Sachs says. It could replace a quarter of work tasks in the US and Europe but may also mean new jobs and a productivity boom. AI has the potential to automate and simplify numerous repetitive and manual tasks that require less creativity and critical reasoning. This can enable workers to focus on more meaningful and... To give full play to the positive role of artificial intelligence technology in employment, we should improve the social security system, accelerate the process of developing high-end domestic... AI will drive job creation Businesses responding to our survey expect artificial intelligence to be a net job creator in the coming five years. Nearly half (49%) of companies expect adopting AI to create jobs, well ahead of the 23% of respondents who expect it to displace jobs. Kweilin Ellingrud: Two influences have changed things. One is we'